In [1]:
#INSTALLATIONS
#!pip install neuralcoref
#!pip install torch torchvision torchaudio --quiet

!pip install transformers[torch] --quiet
#!pip install accelerate --quiet
!pip install datasets --quiet
!pip install rouge_score --quiet
!pip install accelerate>=0.20.1 --quiet
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
!pip install sentence_transformers --quiet
!pip install bert_score --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
#GENERAL SUMMARY
from transformers import pipeline

#SUMMARIZE ACTION ITEMS
summaries = []
summarizer = pipeline("summarization", model="lidiya/bart-large-xsum-samsum")
#knkarthick/bart-large-xsum-samsum
#knkarthick/MEETING-SUMMARY-BART-LARGE-XSUM-SAMSUM-DIALOGSUM

def general_sum(text, flag = False, proportion = 3):
  tokenizer = summarizer.tokenizer
  tokenized_text = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
  input_length = tokenized_text.input_ids.size(1)
  if flag == False:
    res = summarizer(text, min_length=1, max_length=input_length-1)
    text_res = res[0]['summary_text']

  else:
    res = summarizer(text, min_length=round(input_length / proportion), max_length=input_length-1)
    text_res = res[0]['summary_text']

  return text_res

In [4]:
#EXTRACT ACTION ITEMS
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from nltk.tokenize import sent_tokenize


def action_item_extraction(text):
    #Load the model and the tokenizer
    model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/reu2023/action_item_classification/models')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    actions = ''
    sentences = sent_tokenize(text)
    for index, sentence in enumerate(sentences):
        # Tokenize the text
        inputs = tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")

        # Feed the tokenized text into the model
        with torch.no_grad():
            outputs = model(**inputs)

        # Raw output to probabilities
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

        # Interpret the model's output
        _, predicted_class = torch.max(probabilities, dim=1)

        if predicted_class.item() == 1:
            surrounding = ''
            start_index = max(0, index - 3)  # Start index for the surrounding sentences
            end_index = min(len(sentences), index + 3)  # End index for the surrounding sentences

            for surrounding_index in range(start_index, end_index):
                surrounding += sentences[surrounding_index]
            actions += general_sum(surrounding)
    return actions



In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer
import re


def make_chunks(text, similarity_threshold=0):
    # split by speaker
    sentences = re.split(r"(?=<[^>]+>)", text)
    #sentences = text.split('\n\n')
    # Initialize the SentenceTransformer model
    #model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    #model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    # Initialize the Hugging Face summarization tokenizer that the BART will use
    tokenizer = summarizer.tokenizer

    # Generate embeddings for all sentences
    embeddings = model.encode(sentences)

    # Initialize the list of processed sentences
    processed_sentences = [sentences[0]]

    for i in range(1, len(sentences)):
        # Calculate cosine similarity between the current sentence and the last sentence in processed_sentences
        similarity = cosine_similarity(
            [embeddings[i]],
            [embeddings[len(processed_sentences)-1]]
        )[0][0]
        #print(similarity)

        # Check the token length with BERT's tokenizer
        new_sentence = processed_sentences[-1] + " " + sentences[i]
        new_sentence_tokens = tokenizer(new_sentence, truncation=True, padding=True, return_tensors="pt")
        new_length = new_sentence_tokens.input_ids.size(1)

        # If the sentences are similar enough and the concatenated sentence will not exceed 1024 tokens, concatenate them
        # Note: Here we use a 1000 token limit to play it safe :)
        if similarity > similarity_threshold and new_length <= 1000:
            processed_sentences[-1] = new_sentence
        # Otherwise, start a new topic
        else:
            processed_sentences.append(sentences[i])
            #print(new_sentence)

    return processed_sentences




In [6]:
def make_chunks_complex_cos(text, similarity_threshold=0):
    # split by speaker
    sentences = re.split(r"(?=<[^>]+>)", text)

    # Initialize the SentenceTransformer model
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

    # Initialize the Hugging Face summarization tokenizer
    tokenizer = summarizer.tokenizer

    # Initialize the list of processed chunks
    processed_chunks = [sentences[0]]

    for i in range(1, len(sentences)):
        # Generate embeddings for all sentences in processed_chunks and the next sentence
        chunk_embedding = model.encode([processed_chunks[-1]])[0]
        next_sentence_embedding = model.encode([sentences[i]])[0]
        # Calculate cosine similarity between the current chunk and the next sentence
        similarity = cosine_similarity(
            [chunk_embedding],
            [next_sentence_embedding]
        )[0][0]
        # print(similarity)
        # Check the token length with BERT's tokenizer
        new_chunk = processed_chunks[-1] + " " + sentences[i]
        new_chunk_tokens = tokenizer(new_chunk, truncation=True, padding=True, return_tensors="pt")
        new_length = new_chunk_tokens.input_ids.size(1)

        # If the sentences are similar enough and the concatenated sentence will not exceed 1024 tokens, concatenate them
        if similarity > similarity_threshold and new_length <= 1000:
            processed_chunks[-1] = new_chunk
        # Otherwise, start a new topic
        else:
            processed_chunks.append(sentences[i])

    return processed_chunks



In [7]:
def postProcessText(text):
  text = text.replace('#', '')
  return text

In [8]:
def preProcessText(text):
  text = text.replace('PM', 'Product Manager')
  text = text.replace('ID', 'Industrial Designer')
  text = text.replace('UI', 'User Interface Designer')
  text = text.replace('ME', 'Marketing Expert')
  return text

In [9]:
from datasets import load_dataset

# Loading the dataset
dataset = load_dataset("TalTechNLP/AMIsum")

# The dataset is now loaded and can be used for your task.
# For example, you can inspect the first instance in the training set like this:
testTranscript = dataset["test"][19]['transcript']
refSum = dataset["test"][19]['summary']
#print(testTranscript)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
def make_chunks_no_cos(text):
    # split by speaker
    sentences = re.split(r"(?=<[^>]+>)", text)
    # Initialize the SentenceTransformer model
    #model = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    #model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    # Initialize the Hugging Face summarization tokenizer that the BART will use
    tokenizer = summarizer.tokenizer

    # Generate embeddings for all sentences
    embeddings = model.encode(sentences)

    # Initialize the list of processed sentences
    processed_sentences = [sentences[0]]

    for i in range(1, len(sentences)):

        # Check the token length with BERT's tokenizer
        new_sentence = processed_sentences[-1] + " " + sentences[i]
        new_sentence_tokens = tokenizer(new_sentence, truncation=True, padding=True, return_tensors="pt")
        new_length = new_sentence_tokens.input_ids.size(1)

        # If the sentences are similar enough and the concatenated sentence will not exceed 1024 tokens, concatenate them
        # Note: Here we use a 1000 token limit to play it safe :)
        if new_length <= 1000:
            processed_sentences[-1] = new_sentence
        # Otherwise, start a new topic
        else:
            processed_sentences.append(sentences[i])
            #print(new_sentence)

    return processed_sentences

In [11]:

def makeChunksSentences(text, similarity_threshold=0):
    # split by sentence
    sentences = nltk.sent_tokenize(text)
    # Initialize the SentenceTransformer model
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    #model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    # Initialize the Hugging Face summarization tokenizer that the BART will use
    tokenizer = summarizer.tokenizer

    # Generate embeddings for all sentences
    embeddings = model.encode(sentences)

    # Initialize the list of processed sentences
    processed_sentences = [sentences[0]]

    for i in range(1, len(sentences)):
        # Calculate cosine similarity between the current sentence and the last sentence in processed_sentences
        similarity = cosine_similarity(
            [embeddings[i]],
            [embeddings[len(processed_sentences)-1]]
        )[0][0]
        #print(similarity)

        # Check the token length with BERT's tokenizer
        new_sentence = processed_sentences[-1] + " " + sentences[i]
        new_sentence_tokens = tokenizer(new_sentence, truncation=True, padding=True, return_tensors="pt")
        new_length = new_sentence_tokens.input_ids.size(1)

        # If the sentences are similar enough and the concatenated sentence will not exceed 1024 tokens, concatenate them
        # Note: Here we use a 1000 token limit to play it safe
        if similarity > similarity_threshold and new_length <= 1000:
            processed_sentences[-1] = new_sentence
        # Otherwise, start a new topic
        else:
            processed_sentences.append(sentences[i])
            #print(new_sentence)

    return processed_sentences

In [12]:
# make chunks complex cos
def make_chunks_sentences_complex_cos(text, similarity_threshold=0):
    # split by sentence
    sentences = nltk.sent_tokenize(text)

    # Initialize the SentenceTransformer model
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

    # Initialize the Hugging Face summarization tokenizer
    tokenizer = summarizer.tokenizer

    # Initialize the list of processed chunks
    processed_chunks = [sentences[0]]

    for i in range(1, len(sentences)):
        # Generate embeddings for all sentences in processed_chunks and the next sentence
        chunk_embedding = model.encode([processed_chunks[-1]])[0]
        next_sentence_embedding = model.encode([sentences[i]])[0]

        # Calculate cosine similarity between the current chunk and the next sentence
        similarity = cosine_similarity(
            [chunk_embedding],
            [next_sentence_embedding]
        )[0][0]
        # print(similarity)
        # Check the token length with BERT's tokenizer
        new_chunk = processed_chunks[-1] + " " + sentences[i]
        new_chunk_tokens = tokenizer(new_chunk, truncation=True, padding=True, return_tensors="pt")
        new_length = new_chunk_tokens.input_ids.size(1)

        # If the sentences are similar enough and the concatenated sentence will not exceed 1024 tokens, concatenate them
        if similarity > similarity_threshold and new_length <= 1000:
            processed_chunks[-1] = new_chunk
        # Otherwise, start a new topic
        else:
            processed_chunks.append(sentences[i])

    return processed_chunks

# print(make_chunks_sentences_complex_cos(text2))

In [13]:
tokenizer = summarizer.tokenizer
def truncateText(combined):
  if len(tokenizer.tokenize(combined)) > 1000:
    print('ACTIVATED')
    combined = tokenizer.tokenize(combined)[:1000]
    combined = tokenizer.convert_tokens_to_string(combined)
  return combined

In [14]:
#RECURSIVE w/ action and cos
tokenizer = summarizer.tokenizer

#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

def recursive_sum(text, first = True, prop = 3):
  text = preProcessText(text)
  part_sums = ''
  if first is True:
    parts = make_chunks(text)
  else:
    parts = makeChunksSentences(text)
  #print(len(parts))
  for part in parts:
    #print(part)
    #print(part)
    gen_sum = general_sum(part)
    #print('GEN_SUM: ' + gen_sum)
    if first is True:
      actions = action_item_extraction(part)
      #print('\nACTIONS: ' + actions)
      combined = gen_sum + actions
      # truncate just in case
      combined = truncateText(combined)
      part_sum = general_sum(combined)
      #print('\nPART_SUM: ' + part_sum)
      part_sums += part_sum
    else:
      part_sums += gen_sum
    #print('\nPART_SUMS: ' + part_sums)
  #print('\nrec_sum: ', recursive_sum(part_sums))
  tokenized_text = tokenizer(part_sums, return_tensors='pt')#, padding=True, return_tensors="pt")
  input_length = tokenized_text.input_ids.size(1)
  #print('\nparts:', part_sums)
  print(input_length)
  if input_length > 1000:
    print('\nagain')
    return recursive_sum(part_sums, False)
  elif input_length < 400:
    return part_sums
  else:
    return general_sum(part_sums, True, prop)

#print(compute_metrics([recursive_sum(dataset["test"][0]['transcript'])], [dataset["test"][0]['summary']]))
# final_sum = recursive_sum(text)
# print(final_sum)
# sum1_2 = recursive_sum(text2, prop =2)
# print('\nFINAL_SUM with prop =2:', sum1_2)
# sum1_3 = recursive_sum(text2, prop =3)
# print('\nFINAL_SUM with prop =3:', sum1_3)
# sum1_4 = recursive_sum(text2, prop =4)
# print('\nFINAL_SUM with prop =4:', sum1_4)
# sum1_5 = recursive_sum(text2, prop =5)
# print('\nFINAL_SUM with prop =5:', sum1_5)

# sum2_2 = recursive_sum(text2, prop =2)
# print('\nFINAL_SUM with prop =2:', sum2_2)
# sum2_3 = recursive_sum(text2, prop =3)
# print('\nFINAL_SUM with prop =3:', sum2_3)
# sum2_4 = recursive_sum(text2, prop =4)
# print('\nFINAL_SUM with prop =4:', sum2_4)
# sum2_5 = recursive_sum(text2, prop =5)
# print('\nFINAL_SUM with prop =5:', sum2_5)

# testSum = recursive_sum_base(testTranscript)
# print('testSum:', testSum)


In [15]:
# yes action items. no cos. just chunks
def recursive_sum_no_cos(text, first = True, prop = 3):
  text = preProcessText(text)
  part_sums = ''
  if first is True:
    parts = make_chunks_no_cos(text)
  else:
    parts = makeChunksSentences(text)
  #print(len(parts))
  for part in parts:
    #print(part)
    #print(part)
    gen_sum = general_sum(part)
    #print('GEN_SUM: ' + gen_sum)
    if first is True:
      actions = action_item_extraction(part)
      #print('\nACTIONS: ' + actions)
      combined = gen_sum + actions
      # truncate just in case
      combined = truncateText(combined)
      part_sum = general_sum(combined)
      #print('\nPART_SUM: ' + part_sum)
      part_sums += part_sum
    else:
      part_sums += gen_sum
    #print('\nPART_SUMS: ' + part_sums)
  #print('\nrec_sum: ', recursive_sum(part_sums))
  tokenized_text = tokenizer(part_sums, return_tensors='pt')#, padding=True, return_tensors="pt")
  input_length = tokenized_text.input_ids.size(1)
  #print('\nparts:', part_sums)
  print(input_length)
  if input_length > 1000:
    print('\nagain')
    return recursive_sum_no_cos(part_sums, False)
  elif input_length < 400:
    return part_sums
  else:
    return general_sum(part_sums, True, prop)

In [16]:
#RECURSIVE
tokenizer = summarizer.tokenizer



def recursive_sum_base(text, first = True, prop = 3):
  text = preProcessText(text)
  part_sums = ''
  if first is True:
    parts = make_chunks(text)
  else:
    parts = makeChunksSentences(text)
  # print(len(parts))
  for part in parts:
    #print(part)
    #print(part)
    gen_sum = general_sum(part)
    #print('GEN_SUM: ' + gen_sum)
    part_sums += gen_sum
    #print('\nPART_SUMS: ' + part_sums)
  #print('\nrec_sum: ', recursive_sum(part_sums))
  tokenized_text = tokenizer(part_sums, return_tensors='pt')#, padding=True, return_tensors="pt")
  input_length = tokenized_text.input_ids.size(1)
  #print('\nparts:', part_sums)
  print(input_length)
  if input_length > 1000:
    #print('\nagain')
    return recursive_sum_base(part_sums, False)
  elif input_length < 400:
    return part_sums
  else:
    return general_sum(part_sums, True, prop)
# modelSum = recursive_sum_base(dataset["test"][0]['transcript'])
# print(modelSum)
# print(recursive_sum_base(text))
# print(compute_metrics([modelSum], [dataset["test"][0]['summary']]))
# sum1_3 = recursive_sum_base(text, prop =3)
# print('\nFINAL_SUM with prop =3:', sum1_3)
# sum2_3 = recursive_sum(text2, prop =3)
# print('\nFINAL_SUM with prop =3:', sum2_3)
# testSum = recursive_sum_base(testTranscript)
# print('testSum:', testSum)

In [17]:
# no action items. no cosine similiary. just split by chunks.
def recursive_sum_base_no_cos(text, first = True, prop = 3):
  text = preProcessText(text)
  part_sums = ''
  if first is True:
    parts = make_chunks_no_cos(text)
  else:
    parts = makeChunksSentences(text)
  # print(len(parts))
  for part in parts:
    #print(part)
    #print(part)
    gen_sum = general_sum(part)
    #print('GEN_SUM: ' + gen_sum)
    part_sums += gen_sum
    #print('\nPART_SUMS: ' + part_sums)
  #print('\nrec_sum: ', recursive_sum(part_sums))
  tokenized_text = tokenizer(part_sums, return_tensors='pt')#, padding=True, return_tensors="pt")
  input_length = tokenized_text.input_ids.size(1)
  #print('\nparts:', part_sums)
  print(input_length)
  if input_length > 1000:
    #print('\nagain')
    return recursive_sum_base_no_cos(part_sums, False)
  elif input_length < 400:
    return part_sums
  else:
    return general_sum(part_sums, True, prop)

In [18]:

#NAIVE TOPIC SPLITTER. split by tokens
def naiveSplitter(text):
    tokenizer = summarizer.tokenizer
    tokens = tokenizer.tokenize(text)
    processed_texts = []
    i = 0
    while i < len(tokens):
        if i + 1000 < len(tokens):
            end = i + 1000
        else:
            end = len(tokens)
        chunk_text = tokenizer.convert_tokens_to_string(tokens[i:end])
        # remove unnecessary spaces
        chunk_text = re.sub(r'\s([?.!,"](?:\s|$))', r'\1', chunk_text)
        processed_texts.append(chunk_text)
        i = end
    return processed_texts

#print(len(naiveSplitter(text)))
# for subset in naiveSplitter(text):
#   tokenizer = summarizer.tokenizer
#   tokens = tokenizer(subset, truncation=True, padding=True, return_tensors="pt")
#   # print(tokens.input_ids.size(1))
#   # print(subset)

In [19]:
#NAIVE PIPELINE
tokenizer = summarizer.tokenizer
def naiveRecSumBase(text, prop = 3):
  text = preProcessText(text)
  part_sums = ''
  parts = naiveSplitter(text)
  #print(len(parts))
  for part in parts:
    #print(part)
    #gen_sum = general_sum(part)
    #print('GEN_SUM: ' + gen_sum)
    #actions = action_item_extraction(part)
    #print('\nACTIONS: ' + actions)
    part_sum = general_sum(part)# + actions)
    #print('\nPART_SUM: ' + part_sum)
    part_sums += part_sum
    #print('\nPART_SUMS: ' + part_sums)
  #print('\nrec_sum: ', recursive_sum(part_sums))
  tokenized_text = tokenizer(part_sums, truncation=True, padding=True, return_tensors="pt")
  input_length = tokenized_text.input_ids.size(1)
  #print('parts:', part_sums)
  #print(input_length)
  if input_length > 1000:
    return naiveRecSumBase(part_sums)
  elif input_length < 400:
    return part_sums
  else:
    return general_sum(part_sums, True, prop)
#final_sum = recursive_sum(text)
# sum1_2Naive = naiveRecSum(text, prop =2)
# print('\nFINAL_SUM with prop =2:', sum2)
# sum1_3Naive = naiveRecSum(text, prop =3)
# print('\nFINAL_SUM with prop =3:', sum3)
# sum1_4Naive = naiveRecSum(text, prop =4)
# print('\nFINAL_SUM with prop =4:', sum4)
# sum1_5Naive = naiveRecSum(text, prop =5)
# print('\nFINAL_SUM with prop =5:', sum5)





In [20]:
def naiveRecSumAction(text, first = True, prop = 3):
  text = preProcessText(text)
  part_sums = ''
  parts = naiveSplitter(text)
  # print(len(parts))
  for part in parts:
    #print(part)
    #print(part)
    gen_sum = general_sum(part)
    #print('GEN_SUM: ' + gen_sum)
    if first is True:

      actions = action_item_extraction(part)
      #print('\nACTIONS: ' + actions)
      combined = gen_sum + actions
      combined = truncateText(combined)
      part_sum = general_sum(combined)
      # Truncate if more than 1000 tokens
      #print('\nPART_SUM: ' + part_sum)
      part_sums += part_sum
    else:
      part_sums += gen_sum
    #print('\nPART_SUMS: ' + part_sums)
  #print('\nrec_sum: ', recursive_sum(part_sums))
  tokenized_text = tokenizer(part_sums, return_tensors='pt')#, padding=True, return_tensors="pt")
  input_length = tokenized_text.input_ids.size(1)
  #print('\nparts:', part_sums)
  #print(input_length)
  if input_length > 1000:
    #print('\nagain')
    return naiveRecSumAction(part_sums, False)
  elif input_length < 400:
    return part_sums
  else:
    return general_sum(part_sums, True, prop)
#print(naiveRecSumAction(dataset["test"][0]['transcript']))

In [21]:
# no action items. no cosine similiary. just split by chunks.
def recursive_sum_base_complex_cos(text, first = True, prop = 3):
  text = preProcessText(text)
  part_sums = ''
  if first is True:
    parts = make_chunks_complex_cos(text)
  else:
    parts = make_chunks_sentences_complex_cos(text)
  # print(len(parts))
  for part in parts:
    #print(part)
    #print(part)
    gen_sum = general_sum(part)
    #print('GEN_SUM: ' + gen_sum)
    part_sums += gen_sum
    #print('\nPART_SUMS: ' + part_sums)
  #print('\nrec_sum: ', recursive_sum(part_sums))
  tokenized_text = tokenizer(part_sums, return_tensors='pt')#, padding=True, return_tensors="pt")
  input_length = tokenized_text.input_ids.size(1)
  #print('\nparts:', part_sums)
  print(input_length)
  if input_length > 1000:
    #print('\nagain')
    return recursive_sum_base_complex_cos(part_sums, False)
  elif input_length < 400:
    return part_sums
  else:
    return general_sum(part_sums, True, prop)

In [22]:
# yes action items. no cos. just chunks
def recursive_sum_complex_cos(text, first = True, prop = 3):
  text = preProcessText(text)
  part_sums = ''
  if first is True:
    parts = make_chunks_complex_cos(text)
  else:
    parts = make_chunks_sentences_complex_cos(text)
  #print(len(parts))
  for part in parts:
    #print(part)
    #print(part)
    gen_sum = general_sum(part)
    #print('GEN_SUM: ' + gen_sum)
    if first is True:
      actions = action_item_extraction(part)
      #print('\nACTIONS: ' + actions)
      combined = gen_sum + actions
      # truncate just in case
      combined = truncateText(combined)
      part_sum = general_sum(combined)
      #print('\nPART_SUM: ' + part_sum)
      part_sums += part_sum
    else:
      part_sums += gen_sum
    #print('\nPART_SUMS: ' + part_sums)
  #print('\nrec_sum: ', recursive_sum(part_sums))
  tokenized_text = tokenizer(part_sums, return_tensors='pt')#, padding=True, return_tensors="pt")
  input_length = tokenized_text.input_ids.size(1)
  #print('\nparts:', part_sums)
  print(input_length)
  if input_length > 1000:
    print('\nagain')
    return recursive_sum_complex_cos(part_sums, False)
  elif input_length < 400:
    return part_sums
  else:
    return general_sum(part_sums, True, prop)

In [28]:
# GET EXAMPLES

for i in range(20):
  print('\n')
  text1 = dataset["test"][i]['transcript']

  model_sum1 = postProcessText(recursive_sum_no_cos(text1))
  print(i, 'with action:\n', model_sum1)
  print('\n')
  text2 = dataset["test"][i]['transcript']
  model_sum2 = postProcessText(recursive_sum_base_no_cos(text2))
  print(i, 'no action:\n', model_sum2)





136
0 with action:
 Sarah is the Project Manager and this is their first meeting. They are developing a remote control for dogs, cats and people with disabilities. They will practice with pens and draw their favourite animal on the white board.Marketing Expert wants the designers to leave a space at the bottom for him.The product has a selling price of twenty five Euros. The production cost of the product is twelve fifty. The profit aim is fifty million Euros.Marketing Expert would like to incorporate a menu display into the design of a remote control.The product manager and marketing expert discuss the designMarketing Expert, Product Manager and User Interface Designer will meet in half an hour in their room.


146
0 no action:
 Sarah is the Project Manager and this is their first meeting. They are developing a remote control for dogs, cats and people with disabilities. They will practice with pens and draw their favourite animal on the white board.Product Manager, Industrial Design

In [24]:
from datasets import load_metric
from bert_score import score

def compute_metrics(mySum, refSum):
    rouge = load_metric("rouge")
    rouge_results = rouge.compute(predictions=mySum, references=refSum)
    f1_results = {}
    for key, value in rouge_results.items():
        f1_results[key] = value.mid.fmeasure

    # Calculate the BERTScore
    _, _, F1 = score(mySum, refSum, lang='en', model_type='bert-base-uncased', num_layers=12, verbose=False)

    return f1_results, torch.mean(F1).item()












In [25]:
import random
# Generate unique random integers between 0 and 19
indices = random.sample(range(97), 20)
indices = list(range(1, 20))
#indices
print(indices)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [ ]:

# METRICS ON NO ACTION ITEMS
#MY SEGMENTATION METHOD

def runMetricsBase(prop = 3):

  # build model summaries
  modelSums = []
  for index in indices:
    #print(index)
    transcript = dataset["test"][index]['transcript']
    modelSum = postProcessText(recursive_sum_base(transcript))
    #print('\n', index, modelSum)
    modelSums.append(modelSum)

  # get reference summaries
  refSums = []
  for index in indices:
    refSum = dataset["test"][index]['summary']
    refSums.append(refSum)
    #print('\n', index, compute_metrics([modelSums[index]], [refSums[index]]))
  # print('\nmodelSums:', modelSums)
  # print('\nrefSums:', refSums)
  return compute_metrics(modelSums, refSums)

# run metrics
print('\nbase my prop3:', runMetricsBase(prop = 3))

#NAIVE SEGMENTATION
def runMetricsBaseNaive(prop = 3):
  # Generate three unique random integers between 0 and 19
  #print(indices)
  # build model summaries
  modelSums = []
  for index in indices:

    #print(index)
    transcript = dataset["test"][index]['transcript']
    modelSum = postProcessText(naiveRecSumBase(transcript))
    #print('\n', index, modelSum)
    modelSums.append(modelSum)

  # get reference summaries
  refSums = []
  for index in indices:
    refSum = dataset["test"][index]['summary']
    refSums.append(refSum)
    #print('\n', index, compute_metrics([modelSums[index]], [refSums[index]]))
  # print('\nmodelSums:', modelSums)
  # print('\nrefSums:', refSums)
  return compute_metrics(modelSums, refSums)

# run metrics
print('\nbase naive prop3:', runMetricsBaseNaive(prop = 3))

# JUST CHUNK SEGMENTATION
def run_metrics_base_no_cos(prop = 3):

  # build model summaries
  modelSums = []
  for index in indices:
    #print(index)
    transcript = dataset["test"][index]['transcript']
    modelSum = postProcessText(recursive_sum_base_no_cos(transcript))
    modelSums.append(modelSum)
    #print('\n', index, modelSum)

  # get reference summaries
  refSums = []
  for index in indices:
    refSum = dataset["test"][index]['summary']
    refSums.append(refSum)
    #print('\n', index, compute_metrics([modelSums[index]], [refSums[index]]))
  # print('\nmodelSums:', modelSums)
  # print('\nrefSums:', refSums)
  return compute_metrics(modelSums, refSums)

# run metrics
print('\nno cos prop3 base:', run_metrics_base_no_cos(prop = 3))

# COMPLEX COS
def run_metrics_complex_cos_base(prop = 3):

  # build model summaries
  modelSums = []
  for index in indices:
    #print(index)
    transcript = dataset["test"][index]['transcript']
    modelSum = postProcessText(recursive_sum_base_complex_cos(transcript))
    modelSums.append(modelSum)
    #print('\n', index, modelSum)

  # get reference summaries
  refSums = []
  for index in indices:
    refSum = dataset["test"][index]['summary']
    refSums.append(refSum)
    #print('\n', index, compute_metrics([modelSums[index]], [refSums[index]]))
  # print('\nmodelSums:', modelSums)
  # print('\nrefSums:', refSums)
  return compute_metrics(modelSums, refSums)

# run metrics
print('\n complex cos prop3 base:', run_metrics_complex_cos_base(prop = 3))









329
460
342
160
300
488
392
186
376
276
344
135
266
276
348
273
364
424
471


<ipython-input-24-cdde3d90c473>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


Token indices sequence length is longer than the specified maximum sequence length for this model (10762 > 1024). Running this sequence through the model will result in indexing errors



base my prop3: ({'rouge1': 0.3699053467086343, 'rouge2': 0.07951413594957027, 'rougeL': 0.1926808509417511, 'rougeLsum': 0.1926444018518072}, 0.6446431875228882)

base naive prop3: ({'rouge1': 0.37070762609502417, 'rouge2': 0.08214804085797599, 'rougeL': 0.19079975505113073, 'rougeLsum': 0.19075474704304007}, 0.6388084292411804)
329
470
289
153
281
488
383
171
353
302
344
145
222
276
348
273
252
384
516

no cos prop3 base: ({'rouge1': 0.3624198426072477, 'rouge2': 0.07163438884263884, 'rougeL': 0.18840059355588468, 'rougeLsum': 0.1886516344169799}, 0.6409499645233154)
336
470
289
153
281
488
383
171
353
302
344
145
222
276
348
273
252
384


In [27]:
# METRICS ON YES ACTION ITEMS

def runMetricsAction(prop = 3):
  # build model summaries
  modelSums = []
  for index in indices:
    #print(index)
    transcript = dataset["test"][index]['transcript']
    #print(index)
    modelSum = postProcessText(recursive_sum(transcript))
    #print('\n', index, modelSum)
    modelSums.append(modelSum)

  # get reference summaries
  refSums = []
  for index in indices:
    refSum = dataset["test"][index]['summary']
    refSums.append(refSum)
    #print('\n', index, compute_metrics([modelSums[index]], [refSums[index]]))
  # print('\nmodelSums:', modelSums)
  # print('\nrefSums:', refSums)
  return compute_metrics(modelSums, refSums)

# run metrics
print('\nmy prop3 w/ action:', runMetricsAction(prop = 3))

def runMetricsActionNaive(prop = 3):
  # Generate three unique random integers between 0 and 19
  # build model summaries
  modelSums = []
  for index in indices:
    #print(index)
    transcript = dataset["test"][index]['transcript']
    modelSum = postProcessText(naiveRecSumAction(transcript))
    #print('\n', index, modelSum)
    modelSums.append(modelSum)

  # get reference summaries
  refSums = []
  for index in indices:
    refSum = dataset["test"][index]['summary']
    refSums.append(refSum)
    #print('\n', index, compute_metrics([modelSums[index]], [refSums[index]]))
  # print('\nmodelSums:', modelSums)
  # print('\nrefSums:', refSums)
  return compute_metrics(modelSums, refSums)

# run metrics
print('\nnaive prop3 w/ action:', runMetricsActionNaive(prop = 3))

# JUST CHUNK SEGMENTATION
def run_metrics_action_no_cos(prop = 3):

  # build model summaries
  modelSums = []
  for index in indices:
    #print(index)
    transcript = dataset["test"][index]['transcript']
    modelSum = postProcessText(recursive_sum_no_cos(transcript))
    #print('\n', index, modelSum)
    modelSums.append(modelSum)

  # get reference summaries
  refSums = []
  for index in indices:
    refSum = dataset["test"][index]['summary']
    refSums.append(refSum)
    #print('\n', index, compute_metrics([modelSums[index]], [refSums[index]]))
  # print('\nmodelSums:', modelSums)
  # print('\nrefSums:', refSums)
  return compute_metrics(modelSums, refSums)

# run metrics
print('\nno cos prop3 w/ action:', run_metrics_action_no_cos(prop = 3))

# COMPLEX COS W/ ACTION
def run_metrics_complex_cos_action(prop = 3):

  # build model summaries
  modelSums = []
  for index in indices:
    #print(index)
    transcript = dataset["test"][index]['transcript']
    modelSum = postProcessText(recursive_sum_complex_cos(transcript))
    modelSums.append(modelSum)
    #print('\n', index, modelSum)

  # get reference summaries
  refSums = []
  for index in indices:
    refSum = dataset["test"][index]['summary']
    refSums.append(refSum)
    #print('\n', index, compute_metrics([modelSums[index]], [refSums[index]]))
  # print('\nmodelSums:', modelSums)
  # print('\nrefSums:', refSums)
  return compute_metrics(modelSums, refSums)

# run metrics
print('\n complex cos prop3 action:', run_metrics_complex_cos_action(prop = 3))


270
317
339
214
326
252
347
110
290
255
267
213
286
412
420
278
344
354
393

my prop3 w/ action: ({'rouge1': 0.36339673112736126, 'rouge2': 0.08195938928287605, 'rougeL': 0.19612639255358877, 'rougeLsum': 0.19617339235888556}, 0.644705593585968)

naive prop3 w/ action: ({'rouge1': 0.3693001353421639, 'rouge2': 0.08627632098482581, 'rougeL': 0.20032976460655488, 'rougeLsum': 0.20051708813642197}, 0.6476807594299316)
270
314
314
181
273
252
292
132
292
266
267
151
163
412
420
278
291
315

no cos prop3 w/ action: ({'rouge1': 0.3620643731626272, 'rouge2': 0.07653902669254138, 'rougeL': 0.19403589020978157, 'rougeLsum': 0.19392248191283928}, 0.6407821178436279)
298
314
314
181
273
252
292
132
292
266
267
151
163
412
420
278
291
315
447

 complex cos prop3 action: ({'rouge1': 0.3629049131846022, 'rouge2': 0.07679749386826673, 'rougeL': 0.194105363910667, 'rougeLsum': 0.19407726453618762}, 0.6409351229667664)
